In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [28]:
# Load historical stock data
data = pd.read_csv('Nifty_200_scripts.csv')
data.drop('Date',axis=1,inplace=True)
data.drop('Symbol',axis=1,inplace=True)

In [29]:
print(data.shape)

(728197, 5)


In [30]:
# Define the number of assets
num_assets = len(data.columns)

# Prepare the data for training the model
X = data.values[:-1]  # Use all but the last row as input
y = data.values[1:]   # Shifted by one row as target

In [31]:
# Split the data into training and testing sets
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]


In [32]:
X_train = np.asarray(X_train).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)

In [33]:
# Define the neural network model
def build_model(input_dim, output_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(32, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(output_dim, activation='softmax')
    ])
    return model

In [34]:
# Build the neural network model
model = build_model(num_assets, num_assets)

In [35]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy')

# Train the model
model.fit(X_train, y_train, epochs=100, verbose=0)


In [36]:
# Predict the portfolio allocation for the next time step
last_row = data.values[-1].reshape(1, num_assets)
portfolio_allocation = model.predict(last_row)


1/1 [==============================] - 5s 5s/step


In [37]:
# Normalize the portfolio allocation to ensure it sums up to 1
portfolio_allocation /= np.sum(portfolio_allocation)

In [38]:
# Calculate portfolio returns over the next week
next_week_returns = data.values[-7:]  # Assume 7 days of returns
portfolio_returns = np.dot(next_week_returns, portfolio_allocation.T)

In [40]:
print(next_week_returns)

[[1.415000e+01 1.420000e+01 1.380000e+01 1.385000e+01 6.556760e+07]
 [1.385000e+01 1.420000e+01 1.375000e+01 1.390000e+01 1.051694e+08]
 [2.760000e+02 2.806500e+02 2.738500e+02 2.755000e+02 5.716400e+06]
 [2.760500e+02 2.780000e+02 2.673000e+02 2.708000e+02 6.048800e+06]
 [2.730000e+02 2.763000e+02 2.671000e+02 2.681000e+02 8.165800e+06]
 [2.699000e+02 2.706000e+02 2.630000e+02 2.638000e+02 1.237560e+07]
 [2.635000e+02 2.675000e+02 2.607000e+02 2.619000e+02 1.109690e+07]]


In [39]:
# Calculate total portfolio return over the next week
total_portfolio_return = np.sum(portfolio_returns)

# Print the portfolio allocation and total return
print("Portfolio Allocation:")
for i, asset in enumerate(data.columns):
    print(f"{asset}: {portfolio_allocation[0][i]}")
print("Total Portfolio Return:", total_portfolio_return)

Portfolio Allocation:
Open: nan
High: nan
Low: nan
Close: nan
Volume: nan
Total Portfolio Return: nan
